In [1]:

import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
#data = pd.read_csv(r'/home/supriya/CDS_vectorization_v2x.csv',dtype={'Error_Label':object} )
data = pd.read_csv(r'/home/supriya/Hybrid_v2_v2x.csv',dtype={'Error_Label':object} )


In [2]:
# split labels and return new dataframe
#Create independent and Dependent Features
ERROR_LABEL=6
def split_label_dict(df):
    # Change 'label' to whatever naming used in the original DataFrame.
    temp_df = df['Error_Label'].str.split("", n = -1, expand = True)
    for i in range(1, 19):
        df[f'l{i}'] = temp_df[i]
        df[f'l{i}'] = pd.to_numeric(df[f'l{i}'])
    return df

newdata=split_label_dict(data)
#print(newdata)
#Create independent and Dependent Features
df = pd.DataFrame(data)
new_df = df.drop(['Error_Label','l18','l17','l16','l15','l14','l13','l12','l11','l10','l9','l8','l7','l6','l5','l3','l4','l2','l1', 'ID'],axis=1)
#new_df = data.drop(['Error_Label','ID'],axis=1)
X = new_df
# Store the variable we are predicting 
Y = newdata[f"l{ERROR_LABEL}"]

# Print the shapes of X & Y
print(X.shape,Y.shape)


(67677, 1399) (67677,)


In [3]:

from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=69,stratify=Y)


In [4]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [37]:

## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15, 20,25,30,35],
 "min_child_weight" : [ 1, 3, 5, 7,9,11,15,19,24,29,30 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
    "booster":['gbtree', 'gblinear','dart'],
    "eta":[0.05, 0.10, 0.15, 0.20, 0.25, 0.30,.40,.50,.60,.70,.80,.90],
    "tree_method":['auto', 'exact', 'approx', 'hist', 'gpu_hist'],
        "updater":['grow_colmaker','prune','grow_histmaker','grow_local_histmaker',
               'grow_quantile_histmaker','grow_gpu_hist','sync','refresh'],
"grow_policy":['depthwise', 'lossguide'],
    "lambda":[0,1,2,3,4],
    "alpha":[0,1,2,3,4], "n_estimators":[100,200,300,400,500],
    "sampling_method":['uniform','gradient_based'],
    "subsample":[.01,.02]
}

In [5]:
## Hyper Parameter Optimization

params={
    "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15, 20,25,30,35],
 "min_child_weight" : [ 1, 3, 5, 7,9,11,15,19,24,29,30 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
    "booster":['gbtree', 'gblinear','dart'],
    "eta":[0.05, 0.10, 0.15, 0.20, 0.25, 0.30,.40,.50,.60,.70,.80,.90],
    "tree_method":['auto', 'exact', 'approx', 'hist', 'gpu_hist'],
"grow_policy":['depthwise', 'lossguide'],
    "reg_lambda":[0,1,2,3,4],
    "alpha":[0,1,2,3,4],
    "n_estimators":[100,200,300,400,500],
    "sampling_method":['uniform','gradient_based'],
    "subsample":[.01,.02]
}

In [6]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [7]:

## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
classifier=xgboost.XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [8]:
from datetime import datetime
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  25 | elapsed:    3.3s remaining:   13.2s
[Parallel(n_jobs=-1)]: Done  14 out of  25 | elapsed:   31.8s remaining:   25.0s
[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:  9.0min remaining:   47.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 10.0min finished



 Time taken: 0 hours 14 minutes and 27.96 seconds.


In [9]:
random_search.best_estimator_

XGBClassifier(alpha=2, base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eta=0.15, gamma=0.0,
              gpu_id=-1, grow_policy='lossguide', importance_type='gain',
              interaction_constraints='', learning_rate=0.150000006,
              max_delta_step=0, max_depth=4, min_child_weight=11, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=2, reg_lambda=3,
              sampling_method='uniform', scale_pos_weight=1, subsample=1,
              tree_method='hist', validate_parameters=1, verbosity=None)

In [55]:
random_search.best_params_

{'tree_method': 'exact',
 'sampling_method': 'gradient_based',
 'reg_lambda': 2,
 'n_estimators': 100,
 'min_child_weight': 5,
 'max_depth': 8,
 'grow_policy': 'depthwise',
 'gamma': 0.2,
 'eta': 0.3,
 'colsample_bytree': 0.4,
 'booster': 'dart',
 'alpha': 0}

In [5]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from collections import Counter
smk = SMOTE()
X_res,y_res=smk.fit_sample(X_train.astype('float'),y_train)
print(X_res.shape,y_res.shape)
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

(56424, 1399) (56424,)
Original dataset shape Counter({1: 28212, 0: 19161})
Resampled dataset shape Counter({0: 28212, 1: 28212})


In [6]:
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
classifier=xgboost.XGBClassifier(alpha=1, base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eta=0.2, gamma=0.3,
              gpu_id=-1, grow_policy='depthwise', importance_type='gain',
              interaction_constraints='', learning_rate=0.3, max_delta_step=0,
              max_depth=8, min_child_weight=29,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=1, reg_lambda=1,
              sampling_method='uniform', scale_pos_weight=1, subsample=0.01,
              tree_method='exact', validate_parameters=1, verbosity=None)
classifier.fit(X_res,y_res)
#classifier.fit(X_res,y_res)
# Predicting the test set results
y_predict = classifier.predict(X_test)
print(accuracy_score(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.7029156816390859


col_0,0,1
l6,,
0,5598,2614
1,3418,8674


In [33]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=69)
X_resampled, y_resampled = rus.fit_resample(X_test, y_test)
print(X_resampled.shape, y_resampled.shape )
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_test)))
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

(17078, 501) (17078,)
Original dataset shape Counter({0: 10116, 1: 8539})
Resampled dataset shape Counter({0: 8539, 1: 8539})


In [34]:
# Predicting the test set results
y_predict = classifier.predict(X_resampled)
print(accuracy_score(y_resampled,y_predict))
pd.crosstab(y_resampled,y_predict)

0.6285864855369481


col_0,0,1
l14,,
0,6104,2435
1,3908,4631


In [26]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pickle
results=classification_report(y_test,y_predict)
macro=f1_score(y_test, y_predict, average='macro')
micro=f1_score(y_test, y_predict, average='micro')
weighted=f1_score(y_test, y_predict, average='weighted')
avg=f1_score(y_test, y_predict, average=None)

accuracy_score=accuracy_score(y_test,y_predict)
cm = confusion_matrix(y_test, y_predict)
roc=roc_auc_score(y_test,y_predict)

print(cm)

print(results) 
print(roc)
#print(accuracy_score)


[[7216 2900]
 [3908 4631]]
              precision    recall  f1-score   support

           0       0.65      0.71      0.68     10116
           1       0.61      0.54      0.58      8539

    accuracy                           0.64     18655
   macro avg       0.63      0.63      0.63     18655
weighted avg       0.63      0.64      0.63     18655

0.6278302965608313
